In [13]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import time
import os, requests

In [ ]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    
    # clck login
    try:
        # log_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            # driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
            time.sleep(2)
            for i in range(2):
                driver.back()
    except Exception as e:
        print(f"No check tab found and the error is {e}") 
        
        
     # number of cars
    try:
        # time.sleep(1)
        cars = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="pagination-pages"]/span[last()]'))).text.strip("()")
        if cars:
            # print(cars)
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[0])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
        
    
    
    # result
    results = []
    car_count = 0
    while car_count < 35: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}  
                    
                    
                    # car details complete
                    try:
                        det_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="order-lg-2 col-12 col-lg-5 auction-details"]'))) 
                        if det_card:
                            
                            # title and loc --> done
                            try:
                                title =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]'))).text.strip()
                                if title:
                                    details['Title'] = title.split("\n")
                                else:
                                    details['Title'] = 'na'
                                
                                loc =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="auction-location"]'))).text.strip()
                                if loc:
                                    details['Location'] = loc
                                else:
                                    details['Location'] = 'na'
                                        
                            except Exception as e:
                                print("No veh title and location found")

                            # main agent --> done
                            try:
                                agent = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]'))) 
                                if agent:
                                    elements = agent.find_elements(By.TAG_NAME, "p")
                                    if elements:
                                        for ele in elements:
                                            agent_lbl = ele.find_element(By.TAG_NAME, 'strong')
                                            agent_val = ele.text.replace(agent_lbl.text, "").strip()
                                            if agent_lbl and agent_val:
                                                details[agent_lbl.text.strip()] = agent_val
                                            else:
                                                print("No agent data found")
                            except Exception as e:
                                print(" No agent card")
                                
                            # pricing --> done
                            try:
                                pricing_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-list"]'))) 
                                if pricing_card:
                                    pri_elements = pricing_card.find_elements(By.TAG_NAME, "p")
                                    if pri_elements:
                                        for pri_ele in pri_elements:
                                            pri_lbl = pri_ele.find_element(By.TAG_NAME, 'strong')
                                            pri_val = pri_ele.text.replace(pri_lbl.text, "").strip()
                                            if pri_lbl and pri_val:
                                                details[pri_lbl.text.strip()] = pri_val
                                            else:
                                                print("No pricing found")
                            except Exception as e:
                                print(" No pricing card")
                                    
                            
                            # pricing footer --> done
                            try:
                                footer_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-footer"]'))) 
                                if footer_card:
                                    fot_ele = footer_card.find_element(By.TAG_NAME, "p")
                                    if fot_ele:
                                        # for pri_ele in pri_elements:
                                        fot_lbl = fot_ele.find_element(By.TAG_NAME, 'strong')
                                        fot_val = fot_ele.text.replace(fot_lbl.text, "").strip()
                                        if fot_lbl and fot_val:
                                            details[fot_lbl.text.strip()] = fot_val
                                        else:
                                            print("No footer pricing found")
                            except Exception as e:
                                print(" No footer pricing card")
                            
                            # inspec rep --> done
                            try:
                                inspec = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="auction-download"]'))).get_attribute("href")
                                if inspec:
                                    details['Inspec_rep'] = inspec
                                else:
                                    details['Inspec_rep'] = 'na'
                            except Exception as e:
                                print("No inspec rep")                    
                    except Exception as e:
                        print("No main details card")
                        
                    # images --> done
                    try:
                        img_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="ug-thumbs-strip-inner"]'))) 
                        if img_card:
                            imgs =WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))) 
                            if imgs:
                                imgs_lst = [img.get_attribute("src") for img in imgs]
                                details['Images'] = ", ".join(imgs_lst)
                            else:
                                print("No images")
                    except Exception as e:
                        print(f"No image card {e}")
                        
                    
                    # veh summary and inspection
                    try:
                        main_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-tabs"]')))
                        if main_card:
                            # veh summary --> done
                            try:
                                veh_sum_btn =WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="summary-tab"]'))) 
                                if veh_sum_btn:
                                    driver.execute_script("arguments[0].scrollIntoView();", veh_sum_btn)
                                    veh_sum_btn.click()
                                    
                                    veh_cont_card = WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="vehicleTabContent"]')))
                                    if veh_cont_card:
                                        
                                        # additional info --> done
                                        try:
                                            add_info = veh_cont_card.find_element(By.TAG_NAME, "p").text.strip()
                                            if add_info:
                                                details['Additional_info'] = add_info
                                            else:
                                                details['Additional_info'] = 'na'
                                        except Exception as e:
                                            print("No add info")
                                        
                                        # other info --> done
                                        try:       
                                            veh_sum = WebDriverWait(veh_cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-summary"]')))
                                            if veh_sum:
                                                veh_trs = WebDriverWait(veh_sum, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                                if veh_trs:
                                                    for veh_tr in veh_trs:
                                                        veh_lbl = veh_tr.find_element(By.TAG_NAME, 'td').text.strip()
                                                        veh_val = veh_tr.find_element(By.TAG_NAME, 'strong').text.strip()
                                                        if veh_lbl and veh_val:
                                                            details[veh_lbl] = veh_val
                                                        else:
                                                            print("No veh details")
                                        except Exception as e:
                                            print("No veh summary info")        
                                    else:
                                        print("No veh summary button")
                            except Exception as e:
                                print("No veh summary card")
                            
                            
                            # inspection --> remains
                            try:
                                inspec_btn =WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="spec-tab"]'))) 
                                if inspec_btn:
                                    driver.execute_script("arguments[0].scrollIntoView();", inspec_btn)
                                    inspec_btn.click()
                                    time.sleep(3)
                                    
                                    # 1) interior
                                    inspec_inter = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[1]')))
                                    # inspec_body = inspec_inter.find_element(By.TAG_NAME, 'tbody')
                                    if inspec_inter:
                                        # label
                                        inspec_lbl = inspec_inter.find_element(By.TAG_NAME, 'h4').text.strip()
                                        
                                        # head data
                                        inspec_head = inspec_inter.find_element(By.TAG_NAME, 'thead')
                                        head_tr = inspec_head.find_element(By.TAG_NAME, 'tr')
                                        if head_tr:
                                        # for head_tr in head_trs:
                                            head_lbls = head_tr.find_elements(By.TAG_NAME, 'th')
                                            head_type = head_lbls[0].text.strip()
                                            head_cond = head_lbls[1].text.strip()
                                        else:
                                            print("No head tr for interior")
                                        
                                        # tbody data
                                        inspec_tbody = inspec_inter.find_element(By.TAG_NAME, 'tbody')
                                        tbody_lst = []
                                        if inspec_tbody:
                                            inspec_trs = inspec_tbody.find_elements(By.TAG_NAME, 'tr')
                                            if inspec_trs:
                                                for inspec_tr in inspec_trs:
                                                    inspec_type_val = inspec_tr.find_element(By.TAG_NAME, 'th').text.strip()
                                                    inspec_cond_val = inspec_tr.find_element(By.TAG_NAME, 'td').text.strip()
                                                    tbody_lst.append({head_type: inspec_type_val, head_cond: inspec_cond_val})
                                        details[inspec_lbl]= tbody_lst            
                                        
                                        
                                    # 2) tyres
                                    inspec_tyre = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]')))
                                    # inspec_body = inspec_inter.find_element(By.TAG_NAME, 'tbody')
                                    if inspec_tyre:
                                        # label
                                        tyre_lbl = inspec_tyre.find_element(By.TAG_NAME, 'h4').text.strip()
                                        
                                        # head data
                                        tyre_head = inspec_tyre.find_element(By.TAG_NAME, 'thead')
                                        tyre_head_tr = tyre_head.find_element(By.TAG_NAME, 'tr')
                                        if head_tr:
                                        # for head_tr in head_trs:
                                            tyre_head_lbls = tyre_head_tr.find_elements(By.TAG_NAME, 'th')
                                            tyre_head_type = tyre_head_lbls[0].text.strip()
                                            tyre_head_make = tyre_head_lbls[1].text.strip()
                                            tyre_head_cond = tyre_head_lbls[2].text.strip()
                                        else:
                                            print("No head tr for interior")
                                        
                                        # tbody data
                                        tyre_tbody = inspec_tyre.find_element(By.TAG_NAME, 'tbody')
                                        tyre_tbody_lst = []
                                        if tyre_tbody:
                                            tyre_trs = tyre_tbody.find_elements(By.TAG_NAME, 'tr')
                                            if inspec_trs:
                                                for tyre_tr in tyre_trs:
                                                    tyre_vals = tyre_tr.find_elements(By.TAG_NAME, 'td')
                                                    tyre_type_val = tyre_vals[0].text.strip()
                                                    tyre_make_val = tyre_vals[1].text.strip()
                                                    tyre_cond_val = tyre_vals[2].text.strip()
                                                    tyre_tbody_lst.append({tyre_head_type: tyre_type_val, tyre_head_make:tyre_make_val ,tyre_head_cond: tyre_cond_val})
                                        details[tyre_lbl]= tyre_tbody_lst 
                                    
                            except Exception as e:
                                print(f"No isnpec card {e}")
                                
                    except Exception as e:
                        print("No buttons found")
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
        
                except Exception as e:
                    print("No cars found")        

            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Load More"]')))
                    if next_link:
                        driver.execute_script("arguments[0].scrollIntoView();", next_link)
                        time.sleep(3)
                        next_link.click()
                        
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
                        print("Hit the next button")
                        # break
                except Exception as e:
                    print("No next found")
                    break
     
        except Exception as e:
            print("No cars found")
            break
        
    df = pd.DataFrame.from_dict(results)
    df.to_csv("cca.csv", index=False)
    driver.quit()
path = "https://www.centralcarauctions.com/auction/197"
scarpe(path)

103 number of cars found
No agent data found
No veh details
No agent data found
No agent data found
No add info
No veh details
No agent data found
No agent data found
No agent data found
No agent data found
No agent data found
No add info
No agent data found
No add info
No veh details
No agent data found
No add info
No agent data found
No agent data found
 No footer pricing card
No agent data found
No add info
No veh details
No agent data found
No add info
No agent data found
No add info
No agent data found
No add info
No agent data found
No agent data found
No add info
No agent data found
No add info
No agent data found
No veh details
No agent data found
No add info
No agent data found
No agent data found
No agent data found
No veh details
No agent data found
No add info
No agent data found
No add info
No agent data found
No veh details
No agent data found
 No footer pricing card
No veh details
No agent data found
No add info
No agent data found
Hit the next button
No agent data found

In [53]:
df = pd.read_csv("cca.csv")
# df['Title'][0]
df.columns

Index(['Title', 'Location', 'Sale date:', 'Sale time:', 'Order:',
       'Lot number:', 'Cap Clean:', 'Cap Avg.:', 'Cap Retail:',
       'Autotrader Retail Value:', 'Inspec_rep', 'Images', 'Additional_info',
       'Mileage:', 'Registration:', 'Reg year:', 'MOT:', 'VAT:', 'V5:',
       'Owners:', 'Transmission:', 'Fuel:', 'Body:', 'Engine size:', 'Colour:',
       'Interior', 'Tyres', 'Service History:', 'Service Stamps:', 'Doors:'],
      dtype='object')

### downloading

In [54]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[['Registration:', 'Images']]


def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Registration:"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Downloaded: Images\CA15USH\CA15USH_1_1738575763_tpzfbs---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_2_1738575763_k2tjJb---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_3_1738575763_cb0GMD---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_4_1738575763_26weon---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_5_1738575763_7w9Ilx---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_6_1738575764_sOclTl---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_7_1738575764_hVjaMM---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_8_1738575764_ICf27A---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_9_1738575764_7qJ1S8---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_10_1738575764_ZVh27s---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_11_1738575765_Xtgwei---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_12_1738575765_aJVsoD---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_13_1738575765_9xjrQd---150-100.jpg
Downloaded: Images\CA15USH\CA15USH_14_1738575765_n8vFMl---150-100.jpg
Downloaded: Images\CA15USH\CA